# Mask R-CNN for Disease Detection and Segmentation

In [1]:
import os
import xml.etree.ElementTree as ET
import numpy as np
from numpy import zeros, asarray

import mrcnn.utils
import mrcnn.config
import mrcnn.model
import tensorflow as tf

!pip list

Using TensorFlow backend.


Package                       Version
----------------------------- --------------
absl-py                       0.15.0
alabaster                     0.7.13
anyio                         4.1.0
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.3.0
asttokens                     2.4.1
astunparse                    1.6.3
async-lru                     1.0.3
atomicwrites                  1.4.1
attrs                         23.2.0
Babel                         2.14.0
backcall                      0.2.0
beautifulsoup4                4.12.3
black                         24.4.2
bleach                        6.1.0
cachetools                    4.2.4
certifi                       2024.2.2
cffi                          1.16.0
charset-normalizer            3.3.2
click                         8.1.7
colorama                      0.4.6
comm                          0.2.2
contourpy                     1.1.1
curio                         1.6
cycler 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Define the `DiseaseDataset` Class

In [2]:
class DiseaseDataset(mrcnn.utils.Dataset):

    def load_dataset(self, dataset_dir, is_train=True):
        self.add_class("dataset", 1, "disease")
        
        images_dir = os.path.join(dataset_dir, 'images')
        annotations_dir = os.path.join(dataset_dir, 'annots')
        
        all_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        all_ids = [os.path.splitext(fname)[0] for fname in all_files]
        all_ids = sorted(all_ids)  # Ensure consistent order

        split_index = int(0.8 * len(all_ids))  # 80%-20% train-val split
        if is_train:
            ids = all_ids[:split_index]
        else:
            ids = all_ids[split_index:]

        for image_id in ids:
            img_path = os.path.join(images_dir, image_id + '.jpg')
            ann_path = os.path.join(annotations_dir, image_id + '.xml')

            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # Loads the binary masks for an image.
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        path = info['annotation']
        boxes, w, h = self.extract_boxes(path)
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('disease'))
        return masks, asarray(class_ids, dtype='int32')

    # A helper method to extract the bounding boxes from the annotation file
    def extract_boxes(self, filename):
        tree = ET.parse(filename)

        root = tree.getroot()

        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)

        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

## Define the `DiseaseConfig` Class

In [3]:
class DiseaseConfig(mrcnn.config.Config):
    NAME = "disease_cfg"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2

    STEPS_PER_EPOCH = 588

## Prepare the Datasets

In [4]:
dataset_root_path = os.path.join(os.getcwd(), 'data')

# Train
train_dataset = DiseaseDataset()
train_dataset.load_dataset(dataset_dir=dataset_root_path, is_train=True)
train_dataset.prepare()

# Validation
validation_dataset = DiseaseDataset()
validation_dataset.load_dataset(dataset_dir=dataset_root_path, is_train=False)
validation_dataset.prepare()

## Metrics per epoch

In [5]:
def calculate_mean_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [6]:
class MetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'Epoch {epoch + 1} Metrics:')
        
        # Calculate mean IoU for validation data
        val_iou = []
        for image_id in validation_dataset.image_ids:
            image = validation_dataset.load_image(image_id)
            mask, _ = validation_dataset.load_mask(image_id)
            results = model.detect([image], verbose=0)
            pred_mask = results[0]['masks']
            iou = calculate_mean_iou(mask, pred_mask)
            val_iou.append(iou)
        
        mean_iou_value = np.mean(val_iou)
        print(f'mean_iou: {mean_iou_value:.4f}')
        
        # Print other metrics
        for metric_name, metric_value in logs.items():
            print(f'{metric_name}: {metric_value:.4f}')


## Configure and Train the Model

In [7]:
# Model Configuration
disease_config = DiseaseConfig()

# Build the Mask R-CNN Model Architecture
model = mrcnn.model.MaskRCNN(mode='training', 
                             model_dir='./', 
                             config=disease_config)

model.load_weights(filepath='mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# Instantiate the custom callback
metrics_callback = MetricsCallback()

# Train the model with the callback
model.train(train_dataset=train_dataset, 
            val_dataset=validation_dataset, 
            learning_rate=disease_config.LEARNING_RATE, 
            epochs=10,  
            layers='heads',
            custom_callbacks=[metrics_callback])



Starting at epoch 0. LR=0.001

Checkpoint Path: ./disease_cfg20240818T1609\mask_rcnn_disease_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (T

c:\Users\Admin\Desktop\dissertation\Mask-RCNN-TF2-Win\.venv\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


KeyboardInterrupt: 

## Save the Trained Model

In [ ]:
model_path = 'Disease_mask_rcnn_trained.h5'
model.keras_model.save_weights(model_path)